<a href="https://colab.research.google.com/github/Ian-Ibrahim/SU-Machine-learning-1222785/blob/main/29thMaySentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.0 MB/s eta 0:00:00


In [ ]:


import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from sklearn.svm import LinearSVC as SVM
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.linear_model import LogisticRegression as LR

from sklearn.metrics import accuracy_score

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
Xtr = dataset['train']['text']
ytr = dataset['train']['label']

Xts = dataset['test']['text']
yts = dataset['test']['label']

Xval = dataset['validation']['text']
yval = dataset['validation']['label']

In [ ]:
texts  = Xtr + Xts + Xval
print(len(texts))

10662


In [ ]:
tok = TfidfVectorizer(max_features = 100)
tok.fit(texts)

train_seq = tok.transform(Xtr).toarray()
test_seq = tok.transform(Xts).toarray()
val_seq = tok.transform(Xval).toarray()

train_lables = np.array(ytr)
test_lables = np.array(yts)
val_lables = np.array(yval)

In [ ]:
# lr
lrclf = LR()
lrclf.fit(train_seq, train_lables)
lrpred = lrclf.predict(test_seq)
print("LR ACC: " , accuracy_score(test_lables, lrpred))

# svm
svmclf = SVM()
svmclf.fit(train_seq, train_lables)
svmpred = svmclf.predict(test_seq)
print("SVM ACC: " , accuracy_score(test_lables, svmpred))


# nb
nbclf = MNB()
nbclf.fit(train_seq, train_lables)
nbpred = nbclf.predict(test_seq)
print("NB ACC: " , accuracy_score(test_lables, nbpred))

LR ACC:  0.6069418386491557
SVM ACC:  0.6078799249530957
NB ACC:  0.6116322701688556


In [ ]:
def rnd():
    preds = []
    for i in range(len(lrpred)):
        if i > 0.5:
            preds.append(1)
        else:
            preds.append(0)

    return preds


preds = rnd()
print("LR ACC: " , accuracy_score(test_lables, preds))

LR ACC:  0.49906191369606


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation = 'relu', input_shape = (train_seq.shape[1], )))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)
hist = model.fit(
    train_seq,
    train_lables,
    validation_data = (val_seq, val_lables),
    epochs = 20,
    batch_size = 16

)


Epoch 1/20
534/534 [==============================] - 4s 5ms/step - loss: 0.6613 - accuracy: 0.6057 - val_loss: 0.6598 - val_accuracy: 0.6201
Epoch 2/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6321 - accuracy: 0.6411 - val_loss: 0.6604 - val_accuracy: 0.6154
Epoch 3/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6246 - accuracy: 0.6499 - val_loss: 0.6596 - val_accuracy: 0.6069
Epoch 4/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6179 - accuracy: 0.6517 - val_loss: 0.6597 - val_accuracy: 0.6098
Epoch 5/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6105 - accuracy: 0.6626 - val_loss: 0.6577 - val_accuracy: 0.6088
Epoch 6/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6030 - accuracy: 0.6676 - val_loss: 0.6560 - val_accuracy: 0.6154
Epoch 7/20
534/534 [==============================] - 2s 3ms/step - loss: 0.5940 - accuracy: 0.6812 - val_loss: 0.6561 - val_accuracy: 0.6144
Epoch 

In [ ]:
loss, accuracy = model.evaluate(test_seq, test_lables)
print(loss)
print(accuracy)

34/34 [==============================] - 0s 5ms/step - loss: 0.6852 - accuracy: 0.6210
0.685196578502655
0.6210131049156189


In [ ]:
tok = TfidfVectorizer(max_features = 1000)
tok.fit(texts)

train_seq = tok.transform(Xtr).toarray()
test_seq = tok.transform(Xts).toarray()
val_seq = tok.transform(Xval).toarray()

train_lables = np.array(ytr)
test_lables = np.array(yts)
val_lables = np.array(yval)

In [ ]:
# lr
lrclf = LR()
lrclf.fit(train_seq, train_lables)
lrpred = lrclf.predict(test_seq)
print("LR ACC: " , accuracy_score(test_lables, lrpred))

# svm
svmclf = SVM()
svmclf.fit(train_seq, train_lables)
svmpred = svmclf.predict(test_seq)
print("SVM ACC: " , accuracy_score(test_lables, svmpred))


# nb
nbclf = MNB()
nbclf.fit(train_seq, train_lables)
nbpred = nbclf.predict(test_seq)
print("NB ACC: " , accuracy_score(test_lables, nbpred))

LR ACC:  0.7410881801125704
SVM ACC:  0.7448405253283302
NB ACC:  0.7448405253283302


In [ ]:
#LogReg
lrclf = LR()
lrclf.fit(train_seq, train_lables)
lrpred = lrclf.predict(test_seq)
print("LR ACC: " , accuracy_score(test_lables, lrpred))

LR ACC:  0.7410881801125704


In [ ]:
def rnd():
    preds = []
    for i in range(len(lrpred)):
        if i > 0.5:
            preds.append(1)
        else:
            preds.append(0)

    return preds


preds = rnd()
print("LR ACC: " , accuracy_score(test_lables, preds))

LR ACC:  0.49906191369606


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation = 'relu', input_shape = (train_seq.shape[1], )))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

hist = model.fit(
    train_seq,
    train_lables,
    validation_data = (val_seq, val_lables),
    epochs = 20,
    batch_size = 16
)

Epoch 1/20
534/534 [==============================] - 3s 4ms/step - loss: 0.6042 - accuracy: 0.6796 - val_loss: 0.5693 - val_accuracy: 0.6942
Epoch 2/20
534/534 [==============================] - 3s 5ms/step - loss: 0.4924 - accuracy: 0.7570 - val_loss: 0.5462 - val_accuracy: 0.7148
Epoch 3/20
534/534 [==============================] - 2s 3ms/step - loss: 0.4622 - accuracy: 0.7744 - val_loss: 0.5485 - val_accuracy: 0.7186
Epoch 4/20
534/534 [==============================] - 2s 3ms/step - loss: 0.4375 - accuracy: 0.7902 - val_loss: 0.5532 - val_accuracy: 0.7242
Epoch 5/20
534/534 [==============================] - 2s 3ms/step - loss: 0.4125 - accuracy: 0.8069 - val_loss: 0.5649 - val_accuracy: 0.7261
Epoch 6/20
534/534 [==============================] - 2s 3ms/step - loss: 0.3849 - accuracy: 0.8238 - val_loss: 0.5808 - val_accuracy: 0.7261
Epoch 7/20
534/534 [==============================] - 2s 3ms/step - loss: 0.3536 - accuracy: 0.8388 - val_loss: 0.6026 - val_accuracy: 0.7251
Epoch 

In [ ]:
loss, accuracy = model.evaluate(test_seq, test_lables)
print(loss)
print(accuracy)

34/34 [==============================] - 0s 3ms/step - loss: 0.8823 - accuracy: 0.7345
0.8823359608650208
0.7345215678215027
